# Google Drive operation  
## 参考サイト  
[AUTOMATIC1111をPaperspace Gradientで実行し， Google Driveに画像を送る（認証にサービスアカウントを利用するVer.）](https://nintech.jp/2023/02/10/automatic1111%E3%82%92paperspace-gradient%E3%81%A7%E5%AE%9F%E8%A1%8C%E3%81%97%EF%BC%8C-google-drive%E3%81%AB%E7%94%BB%E5%83%8F%E3%82%92%E9%80%81%E3%82%8B%EF%BC%88%E8%AA%8D%E8%A8%BC%E3%81%AB%E3%82%B5/)  
  
「Googleの認証」セクション参照  
[PyDriveによりPythonからGoogleドライブを操作する【OAuth使わず】](https://self-development.info/pydrive%E3%81%AB%E3%82%88%E3%82%8Apython%E3%81%8B%E3%82%89google%E3%83%89%E3%83%A9%E3%82%A4%E3%83%96%E3%82%92%E6%93%8D%E4%BD%9C%E3%81%99%E3%82%8B%E3%80%90oauth%E4%BD%BF%E3%82%8F%E3%81%9A%E3%80%91/)  
[gspreadをインストールしてGoogleスプレッドシートをPythonで操作する](https://self-development.info/gspread%e3%82%92%e3%82%a4%e3%83%b3%e3%82%b9%e3%83%88%e3%83%bc%e3%83%ab%e3%81%97%e3%81%a6google%e3%82%b9%e3%83%97%e3%83%ac%e3%83%83%e3%83%89%e3%82%b7%e3%83%bc%e3%83%88%e3%82%92python%e3%81%a7%e6%93%8d/)  

ソースコード参照  
[Google ドライブのファイルを Python でダウンロードする](https://qiita.com/koji4104/items/618c9dad0140cc1558e3)  


# Setup

In [2]:
# setup
%cd /notebooks

!pip install PyDrive
!pip install gdown
!pip install --upgrade gdown
!pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

/notebooks


# Sync GoogleDrive

In [4]:
# fill SHARE_FOLDER_ID
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.errors import HttpError as HTTPError
from httplib2 import Http
import os
import io
import shutil

# pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from googleapiclient.http import MediaIoBaseDownload

SCOPES = ['https://www.googleapis.com/auth/drive']
SHARE_FOLDER_ID = '18iQra2qjW-RUbqMp3boTmT6d8DDPUBHL'

sa_creds = service_account.Credentials.from_service_account_file( 'service_account.json')
scoped_creds = sa_creds.with_scopes(SCOPES)
drive_service = build('drive', 'v3', credentials=scoped_creds)
response = drive_service.files().list( supportsAllDrives=True, includeItemsFromAllDrives=True, q=f"parents in '{SHARE_FOLDER_ID}' and trashed = false", fields="nextPageToken, files(id, name)").execute()

files = response.get('files', [])
for file in response.get('files', []): print(f"Found file: {file.get('name')} ({file.get('id')})")

if not os.path.isdir("/notebooks/GoogleDrive"):
    os.mkdir("/notebooks/GoogleDrive")

if files:
    #Clear directory
    shutil.rmtree("/notebooks/GoogleDrive")
    os.mkdir("/notebooks/GoogleDrive")
    for file in files:
        request = drive_service.files().get_media(fileId=file['id'])
        fh = io.FileIO("/notebooks/GoogleDrive/"+file['name'], mode='wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            _, done = downloader.next_chunk()

Found file: Resize_LoRA_auTomsGRSupra.zip (1K04foW00wwmClANNC_yAYIr7p_8W144Y)
Found file: lora_train_sample_pack.zip (14OnKXcH3dVcsSo4UNJ-U5Pd0eLFGwl0R)
